Construct suburbs out of SA1 areas.

In [1]:
import pandas

In [15]:
import shapefile
from shapely.geometry import shape
from tqdm import tqdm_notebook as tqdm

In [57]:
sa1_shape = shapefile.Reader('../shapesnsw/MB_2016_NSW')
sa1_records = [r for r in sa1_shape.records()]
sa1_shape = [shape(s) if s.shapeType != 0 else None for s in tqdm(sa1_shape.shapes())]

In [61]:
suburb_shape = shapefile.Reader('../NSW_LOCALITY_POLYGON_shp/NSW_LOCALITY_POLYGON_shp')
suburb_records = [r for r in suburb_shape.records()]
suburb_shape = [shape(s) for s in tqdm(suburb_shape.shapes())]

In [104]:
sa1_to_suburb = {}  # SA1_7DIG16 -> LC_PLY_PID
for s, r in zip(tqdm(sa1_shape[1:]), sa1_records[1:]):
    if not 116 <= int(r.SA2_MAIN16[:3]) <= 128:
        continue
    if not s:
        continue
    suburb_to_overlap = {}
    for suburb, subr in zip(suburb_shape, suburb_records):
        if not suburb.overlaps(s):
            continue
        suburb_to_overlap[subr.LC_PLY_PID] = suburb.intersection(s).area
    if not suburb_to_overlap:
        continue
    suburb = max(suburb_to_overlap, key=suburb_to_overlap.get)
    sa1_to_suburb[r.SA1_7DIG16] = suburb

In [96]:
len({r.LC_PLY_PID for r in suburb_records})

4592

In [87]:
r = suburb_records[0]

In [106]:
import json
with open('sa1_to_suburb.json', 'w') as f:
    json.dump(sa1_to_suburb, f)

In [107]:
suburb_names = {r.LC_PLY_PID: r.NSW_LOCA_2 for r in suburb_records}

In [109]:
with open('suburb_names.json', 'w') as f:
    json.dump(suburb_names, f)